## reference source:

- how to get brand list
    - cross reference: Sephora data from Kaggle(link: https://www.kaggle.com/raghadalharbi/all-products-available-on-sephora-website)
- how to get product list 
- how to get review list
(link: https://github.com/Shirleyiscool/Scraping-Sephora)
- link to get proxies (http://www.freeproxylists.net/zh/?c=US&pr=HTTPS&u=80&s=ts)

### steps:
- get brand list
- get product list
    - filter only product with review number > 1000
- get review data for product with review number >1000


In [115]:
pd.options.display.max_rows = 999

### Step1: get brand list

In [ ]:
# Get Brand List
import requests
from bs4 import BeautifulSoup

# Get Response of "brandlist" Website from Sephora
band_lst_link = "https://www.sephora.com/brands-list"
response = requests.get(band_lst_link)

# Use BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

### Step2: get Product list (not full list, maybe dut to proxy issue)

In [ ]:
# Scraping brand links and save them into a list
brand_link_lst = []
main_box = soup.find_all(attrs={"data-comp": "BrandsList StyledComponent BaseComponent "})[0]
for brand in main_box.find_all('li'):
    brand_link_lst.append("https://www.sephora.com" +
                          brand.a.attrs['href']+"/all?pageSize=300")

# Write brand links into a file:
with open('data/brand_link.txt', 'w') as f:
    for item in brand_link_lst:
        f.write(f"{item}\n")

# Indicate scraping completion
print(f'Got All Brand Links! There are {len(brand_link_lst)} brands in total.')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def scape_product(link, proxy=None):
    """
    A function to scape all the product links from a given brand link.
    """
    try:
        response = requests.get(link, proxies={
                                "http": proxy, "https": proxy}, timeout=15)
    except:
        print(f'\r Unsuccessfully get data for {link.split("/")[4]}', end="")
        return None
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    product_link_lst = []
    try:
        product_box = soup.find_all(attrs={"data-comp": "ProductGrid "})[0]
    # There might be no products for that brand
    except IndexError:
        return []
    for product in product_box.find_all('a',
                                        attrs={"data-comp": "ProductItem "}):
        # use function split to remove text like "grid p12345"
        product_link_lst.append(
            "https://www.sephora.com" + product.attrs['href'].split()[0])
    return product_link_lst


# Read brand links file
product_link_dic = {'brand': [], 'product_links': []}
num_lines = sum(1 for line in open("data/brand_link.txt", "r"))

# Scape all the product links from all the brands links.
# This will take some time!
ct = 1

# Get proxies from http://www.freeproxylists.net/zh/?c=US&pr=HTTPS&u=80&s=ts
px = ['143.198.222.22:8080', '143.198.206.183:8080', '157.230.208.88:8080']
px_idx = 0

for brand_link in open("data/brand_link.txt", "r"):
    brand_name = brand_link.split('/')[4]
    product_link_list = scape_product(brand_link[:-1], proxy=px[px_idx])

    # If one proxy does not work, use another
    while product_link_list is None:
        px_idx += 1
        if px_idx == 3:
            px_idx = 0
        product_link_list = scape_product(brand_link[:-1], proxy=px[px_idx])

    print(f'\r === {ct} / {num_lines} ===  {brand_name} === {px[px_idx]}',
          end="")
    product_link_dic['brand'] += [brand_name] * len(product_link_list)
    product_link_dic['product_links'] += product_link_list
    ct += 1

# Write the result into csv file
product_link_df = pd.DataFrame(product_link_dic)
product_link_df.to_csv('data/product_links.csv', index=False)

# Indicate scraping completion
print(f'Got All product Links! There are {len(product_link_df)} products in '
      f'total.')

In [ ]:
import re

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_data(product_link, px_list=None):
    """Get product information"""
    data_dic = {'pd_id': [], 'size_and_item': [], 'category': [],
                'price': [], 'love_count': [], 'reviews_count': []}
    px_idx = 0
    proxy = None if px_list is None else px_list[px_idx]

    while True:
        try:
            response = requests.get(product_link, proxies={
                "http": proxy, "https": proxy}, timeout=15)
        except:
            if px_idx == len(px_list) - 1:
                px_idx = 0
            else:
                px_idx += 1
            proxy = px_list[px_idx]
            continue

        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        data_dic['pd_id'] = re.findall(R'P[0-9]{3,6}', product_link)[0]

        # Get Category
        try:
            cat_box = soup.find_all(attrs={'data-comp': 'ProductBreadCrumbs BreadCrumbs '})[0]
            cat_list = [cat.string for cat in cat_box.find_all('a')]
            category = ', '.join(cat_list)
        except:
            category = None


        category

        # Size and Content
        try:
            size_and_item = soup.find(
                attrs={"data-at": "sku_size_label"}).get_text()
        except:
            size_and_item = None

        size_and_item

        # Get Price
        try:
            price = soup.find_all(attrs={'data-comp': 'Price '})[
                0].get_text()
        except:
            price = None

        price


        # Get love counts
        try:
            love_count = soup.find('span', attrs={
                "class": "css-jk94q9"}).get_text()
        except:
            love_count = None

        love_count


        # review nums
        try:
        #     link_json = soup.find(attrs={"id": "linkJSON"})
        #     json_str = str(link_json)
        #     reviews = re.findall(R'\"reviews\":(.*?)\,', json_str)
            reviews = soup.find('span', attrs = {
        #         'class':"css-nv7myq eanm77i0",
#                 'class': "css-1vj6vps eanm77i0"
        #         'data-comp' : 'StyledComponent BaseComponent ',
        #         'id': 'ratings-reviews-container'
                'data-at': 'number_of_reviews'
            }).get_text()
            reviews_count = reviews
        except:
            reviews_count = None


        data_dic['category'] = category
        data_dic['size_and_item'] = size_and_item
        data_dic['love_count'] = love_count
        data_dic['reviews_count'] = reviews_count
        data_dic['price'] = price
        break
    return data_dic


px_list_ = [
#     '167.99.218.191:8080',
#           '144.217.254.175:3128', 
    '165.225.77.47:9443',
          '54.37.137.211:3128', '165.22.91.197:8080',
          '165.225.77.47:8800', '165.225.77.47:9400',
          '165.225.77.47:80', '165.225.77.47:443',
    '143.198.222.22:8080', '143.198.206.183:8080', '157.230.208.88:8080',
            ]

pd_links_df = pd.read_csv('data/product_links.csv')
product_links = pd_links_df['product_links']

result = []
for i, link in enumerate(product_links[:]):
    result.append(get_data(link, px_list_))
    pd_df = pd.DataFrame(result)
    pd_df.to_csv('data/pd_info.csv', index=False)
    print(f'{i + 1:04d} / {len(product_links)} || {link}')

# Step3: Get Reviews

In [1]:
import re
import time

import pandas as pd
import requests
import json


In [2]:
# Load product link data frame
pd_links_df = pd.read_parquet('./data/sephora_historical_data_april_2020.parquet')
#pd.read_csv('data/product_links.csv')

In [3]:
pd_links_df['product_links'] = pd_links_df['URL']

In [36]:
# remove ones already done
with open("data/scraper_result.json", "r") as file:
    result = json.load(file)

In [37]:
pids = list(result)

In [69]:
pids_cleaned = [i for i in list(result) if result[i]!=[None, None]]

In [70]:
len(pids_cleaned)

1

In [38]:
# pids

In [39]:
pd_links_df['key'] = pd_links_df['URL'].str.findall('P[0-9]{4,7}').apply(lambda x: x[0])

In [40]:
pd_links_df.shape

(9168, 23)

In [41]:
pd_links_df = pd_links_df[~pd_links_df['key'].isin(pids)].copy()

In [4]:
pd_links_df.shape

(9168, 22)

In [ ]:
# Add a column of product id
pd_links_df['pd_id'] = [re.findall('P[0-9]{4,7}', link)[0] for link
                        in pd_links_df['product_links']]


def scrape_reviews(p_id, proxy=None):
    url = 'https://api.bazaarvoice.com/data/reviews.json'
    params = {
        'Filter': f'ProductId:{p_id}',
        'Sort': 'SubmissionTime:desc',
        'Limit': 100,
        'Offset': 0,
        'Include': 'Products,Comments',
        'Stats': 'Reviews',
        'passkey': 'caQ0pQXZTqFVYA1yYnnJ9emgUiW59DXA85Kxry8Ma02HE',
        'apiversion': 5.4,
        'Locale': 'en_US',
    }

    
    reviews = []
    loop = 0

    while True:
        params['Offset'] = len(reviews)

        # Make the same request that Javascript makes
        try:
            r = requests.get(url, params=params, proxies={
                "http": proxy, "https": proxy}, timeout=15)
        except:
            print(f'{proxy} Cannot connect!')
            return None, None
        if loop == 0:
            try:
                product = r.json()['Includes']['Products']
            except KeyError:
                product = []

        # break if we have an error or have all the reviews
        if (r.status_code != 200) or (
                len(reviews) >= r.json()['TotalResults']):
            break

        # add the list of results to current results
        reviews.extend(r.json()['Results'])

        # Give a pause, so we don't get blocked
        time.sleep(0.2)
        loop += 1

    # Show how many reviews we scraped
    print(f'{p_id}: {len(reviews)} reviews')
    time.sleep(0.5)
    return product, reviews


# Scrape Product and Review Data
# result already imported from file
# result = {}

proxies = [ 
          '165.225.77.47:8800', '165.225.77.47:9400',
          '165.225.77.47:80', '165.225.77.47:443',
'167.99.218.191:8080','144.217.254.175:3128', 
           '165.225.77.47:9443','54.37.137.211:3128','165.22.91.197:8080',]
px_id = 0
loop = 0

for pid in pd_links_df['pd_id']:
    loop_ = loop % 1000
    if (loop_ < 900) and (loop_ >= 150):
        product, reviews = None, None
        while True:
            if px_id == len(proxies):
                px_id = 0

            product_data, reviews_data = scrape_reviews(pid,
                                                        proxy=proxies[px_id])
            if product_data is not None:
                break
            px_id += 1

    # Use my own server to connect
    else:
        product_data, reviews_data = scrape_reviews(pid)
    loop += 1

    print(f'{proxies[px_id]} || {loop:04d}/{len(pd_links_df)}')
    result[pid] = [product_data, reviews_data]

json_result = json.dumps(result)
f = open("data/scraper_result.json", "w")
f.write(json_result)
f.close()

In [7]:
json_result = json.dumps(result)
f = open("data/scraper_result.json", "w")
f.write(json_result)
f.close()

In [9]:
len(result)

272

In [110]:
# result['P393151']

In [111]:
# Json format
# {'PID': [ {'PID' :'product_dict'}, 'review_dict']}

In [89]:
product_lst = []
reviews_lst = []
for name, value in result.items():
    if value == [[],[]]:
        pass
        print(name, value)
    else:
        reviews_lst.extend(value[1])
        try:
            product_lst.append(value[0][name])
        except:
            print(name)

P450814 [[], []]
P454768
P457815
P454769
P429259
P456165
P378121
P426155


In [13]:
df_review = pd.DataFrame(reviews_lst)

In [17]:
context = pd.json_normalize(df_review['ContextDataValues'])

In [20]:
context.shape

(38569, 48)

In [15]:
df_review.columns

Index(['UserNickname', 'Rating', 'ReviewText', 'ContextDataValues',
       'ProductId', 'UserEmailAddress', 'Videos', 'CommentIds', 'Photos',
       'TotalNegativeFeedbackCount', 'RatingRange', 'CampaignId',
       'UserLocation', 'SecondaryRatingsOrder', 'TotalCommentCount',
       'ContextDataValuesOrder', 'ClientResponses', 'TagDimensionsOrder',
       'TotalPositiveFeedbackCount', 'IsRatingsOnly', 'LastModificationTime',
       'Pros', 'ProductRecommendationIds', 'BadgesOrder', 'AuthorId',
       'Helpfulness', 'SubmissionTime', 'AdditionalFields', 'Title',
       'AdditionalFieldsOrder', 'TotalFeedbackCount', 'ModerationStatus',
       'IsFeatured', 'Cons', 'TagDimensions', 'SecondaryRatings', 'Badges',
       'SubmissionId', 'IsSyndicated', 'IsRecommended', 'LastModeratedTime',
       'Id', 'ContentLocale'],
      dtype='object')

In [112]:
df_review_final = pd.concat([df_review, context], axis = 1)

In [113]:
df_review_final.shape

(38569, 91)

In [117]:
df_review_final.head()

,UserNickname,Rating,ReviewText,ContextDataValues,ProductId,UserEmailAddress,Videos,CommentIds,Photos,TotalNegativeFeedbackCount,...,age.ValueLabel,age.DimensionLabel,beautyInsider.Value,beautyInsider.Id,beautyInsider.ValueLabel,beautyInsider.DimensionLabel,hairCondition.Value,hairCondition.Id,hairCondition.ValueLabel,hairCondition.DimensionLabel
0,Aquamarinade,2,I bought this because I already love Arancia d...,"{'skinType': {'Value': 'combination', 'Id': 's...",P443401,None,[],[],[],0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adele2090,1,Why is this 82$ when the value is lower? That ...,"{'skinType': {'Value': 'combination', 'Id': 's...",P443401,None,[],[],[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SMDSoCal,4,I had a few of these as samples and really lik...,"{'skinType': {'Value': 'combination', 'Id': 's...",P443401,None,[],[],[],0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EastcoastAmy,3,Fell in love with Acqua Di Parma during a stay...,"{'skinType': {'Value': 'combination', 'Id': 's...",P443401,None,[],[],[],2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,karenleeza,5,I've had this set for a year now and I constan...,"{'hairConcerns': {'Value': 'Hold', 'Id': 'hair...",P443401,None,[],[],[],4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df_review_final.to_csv('./data/review_sample.csv', sep = '\t')

In [92]:
len(product_lst)

264

In [93]:
df_product = pd.DataFrame(product_lst)

In [99]:
df_product['key'] = df_product['FamilyIds'].apply(lambda x: x[0])

In [121]:
df_product.head()

,CategoryId,EANs,Description,ModelNumbers,Attributes,ImageUrl,BrandExternalId,UPCs,ReviewStatistics,Name,...,QuestionIds,FamilyIds,ReviewIds,ProductPageUrl,ISBNs,Id,AttributesOrder,ManufacturerPartNumbers,StoryIds,key
0,cat1910032,[8028713571589],This enchanting set comes in a specially handc...,[],"{'BV_FE_EXPAND': {'Values': [{'Locale': None, ...",https://www.sephora.com/productimages/sku/s221...,5847,[],"{'FeaturedReviewCount': 0, 'RecommendedCount':...",Blu Mediterraneo Mini Eau de Toilette Set,...,[],[P443401],[],https://www.sephora.com/product/blu-mediterran...,[],P443401,"[BV_FE_EXPAND, BV_FE_FAMILY]",[],[],P443401
1,cat60150,"[8028713000089, 8028713000096, 8028713250361]",Fragrance Family: Fresh Scent Type: Fresh Ci...,[],"{'BV_FE_EXPAND': {'Values': [{'Locale': None, ...",https://www.sephora.com/productimages/sku/s204...,5847,[],"{'FeaturedReviewCount': 0, 'RecommendedCount':...",Colonia,...,[],[P163604],[],https://www.sephora.com/product/colonia-P163604,[],P163604,"[BV_FE_EXPAND, BV_FE_FAMILY]",[],[],P163604
2,cat60148,"[8028713570018, 8028713570025, 8028713570261, ...",Fragrance Family: Fresh\tScent Type: Fresh Cit...,[],"{'BV_FE_EXPAND': {'Values': [{'Locale': None, ...",https://www.sephora.com/productimages/sku/s141...,5847,[],"{'FeaturedReviewCount': 0, 'RecommendedCount':...",Arancia di Capri,...,[],[P375388],[],https://www.sephora.com/product/blu-mediterran...,[],P375388,"[BV_FE_EXPAND, BV_FE_FAMILY]",[],[],P375388
3,cat60148,"[8028713570070, 8028713570087, 8028713570292, ...",Fragrance Family: Earthy & Woody Scent Type: F...,[],"{'BV_FE_EXPAND': {'Values': [{'Locale': None, ...",https://www.sephora.com/productimages/sku/s141...,5847,[],"{'FeaturedReviewCount': 0, 'RecommendedCount':...",Mirto di Panarea,...,[],[P307804],[],https://www.sephora.com/product/blu-mediterran...,[],P307804,"[BV_FE_EXPAND, BV_FE_FAMILY]",[],[],P307804
4,cat1910032,[8028713250439],The Colonia Miniature Set comes in an iconic A...,[],"{'BV_FE_EXPAND': {'Values': [{'Locale': None, ...",https://www.sephora.com/productimages/sku/s221...,5847,[],"{'FeaturedReviewCount': 0, 'RecommendedCount':...",Mini Colonia Set,...,[],[P443400],[],https://www.sephora.com/product/colonia-miniat...,[],P443400,"[BV_FE_EXPAND, BV_FE_FAMILY]",[],[],P443400


In [122]:
df_product.to_csv('./data/product_sample.csv', sep = '\t')